In [63]:
# encoding=utf8

# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


import pickle

from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [64]:
#Set de entrenamiento con el target Damage Grade includio
train = pd.read_csv('data/clean_data.csv',dtype = {
    "building_id" : "int32", #0
    "geo_level_1_id" : "int8", #1
    "geo_level_2_id" : "int16", #2
    "geo_level_3_id" : "int16", #3
    "count_floors_pre_eq" : "int8", #4
    "age" : "int16", #5
    "area_percentage" :"int8", #6
    "height_percentage" :"int8", #7
    "land_surface_condition" :"category", #8
    "foundation_type" :"category", #9
    "roof_type" :"category", #10
    "ground_floor_type" :"category",#11
    "other_floor_type" :"category",#12
    "position" :"category",#13
    "plan_configuration" :"category", #14
    "has_superstructure_adobe_mud" : "bool",
    "has_superstructure_mud_mortar_stone" : "bool",
    "has_superstructure_stone_flag" : "bool",
    "has_superstructure_cement_mortar_stone" : "bool",
    "has_superstructure_mud_mortar_brick" : "bool",
    "has_superstructure_cement_mortar_brick" : "bool",
    "has_superstructure_timber" : "bool",
    "has_superstructure_bamboo" : "bool",
    "has_superstructure_rc_non_engineered" : "bool",
    "has_superstructure_rc_engineered" : "bool",
    "has_superstructure_other" : "bool",
    "legal_ownership_status" :"category",# 26
    "count_families" : "int8",
    "has_secondary_use" : "bool",
    "has_secondary_use_agriculture" : "bool",
    "has_secondary_use_hotel" : "bool",
    "has_secondary_use_rental" : "bool",
    "has_secondary_use_institution" : "bool",
    "has_secondary_use_school" : "bool",
    "has_secondary_use_industry" : "bool",
    "has_secondary_use_health_post" : "bool",
    "has_secondary_use_gov_office" : "bool",
    "has_secondary_use_use_police" : "bool",
    "has_secondary_use_other" : "bool",
    "damage_grade":"int8"})
train

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,...,False,False,False,False,False,False,False,False,False,3
1,28830,8,900,2812,2,10,8,7,o,r,...,False,False,False,False,False,False,False,False,False,2
2,94947,21,363,8973,2,10,5,5,t,r,...,False,False,False,False,False,False,False,False,False,3
3,590882,22,418,10694,2,10,6,5,t,r,...,False,False,False,False,False,False,False,False,False,2
4,201944,11,131,1488,3,30,8,9,t,r,...,False,False,False,False,False,False,False,False,False,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,False,False,False,False,False,False,False,False,False,2
260597,669485,17,715,2060,2,0,6,5,t,r,...,False,False,False,False,False,False,False,False,False,3
260598,602512,17,51,8163,3,55,6,7,t,r,...,False,False,False,False,False,False,False,False,False,3
260599,151409,26,39,1851,2,10,14,6,t,r,...,False,False,False,False,False,False,False,False,False,2


In [6]:
#Set de Test
test = pd.read_csv('data/test_values.csv',dtype = {
    "building_id" : "int32",
    "geo_level_1_id" : "int8", 
    "geo_level_2_id" : "int16",
    "geo_level_3_id" : "int16",
    "count_floors_pre_eq" : "int8",
    "age" : "int16",
    "area_percentage" :"int8",
    "height_percentage" :"int8",
    "land_surface_condition" :"category",
    "foundation_type" :"category",
    "roof_type" :"category",
    "ground_floor_type" :"category",
    "other_floor_type" :"category",
    "position" :"category",
    "plan_configuration" :"category",
    "has_superstructure_adobe_mud" : "bool",
    "has_superstructure_mud_mortar_stone" : "bool",
    "has_superstructure_stone_flag" : "bool",
    "has_superstructure_cement_mortar_stone" : "bool",
    "has_superstructure_mud_mortar_brick" : "bool",
    "has_superstructure_cement_mortar_brick" : "bool",
    "has_superstructure_timber" : "bool",
    "has_superstructure_bamboo" : "bool",
    "has_superstructure_rc_non_engineered" : "bool",
    "has_superstructure_rc_engineered" : "bool",
    "has_superstructure_other" : "bool",
    "legal_ownership_status" :"category",
    "count_families" : "int8",
    "has_secondary_use" : "bool",
    "has_secondary_use_agriculture" : "bool",
    "has_secondary_use_hotel" : "bool",
    "has_secondary_use_rental" : "bool",
    "has_secondary_use_institution" : "bool",
    "has_secondary_use_school" : "bool",
    "has_secondary_use_industry" : "bool",
    "has_secondary_use_health_post" : "bool",
    "has_secondary_use_gov_office" : "bool",
    "has_secondary_use_use_police" : "bool",
    "has_secondary_use_other" : "bool"})

In [65]:
y = train["damage_grade"].values
X = train.drop(['damage_grade', 'building_id'], axis=1)
X_test = test.drop(['building_id'], axis=1)
y

array([3, 2, 3, ..., 3, 2, 3], dtype=int8)

In [66]:

#split train set for testing
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=1, stratify=y, shuffle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape

((234540, 38), (26061, 38), (234540,), (26061,), (86868, 38))

## CatBoost

In [12]:
!pip install catboost

In [67]:
cate_features_index = np.where(X.dtypes != float)[0]


In [79]:
cate_features_index = list(range(0,38))
np.array(cate_features_index)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37])

In [17]:
from catboost import Pool, CatBoostClassifier, cv

model = CatBoostClassifier(eval_metric='Accuracy', loss_function='MultiClass',use_best_model=True,random_seed=1).fit(X_train,y_train,cat_features=cate_features_index,eval_set=(X_val,y_val))
print(model.score(X_train, y_train))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.120429
0:	learn: 0.6940181	test: 0.6956372	best: 0.6956372 (0)	total: 4.45s	remaining: 1h 14m 5s
1:	learn: 0.6998764	test: 0.7008173	best: 0.7008173 (1)	total: 7.62s	remaining: 1h 3m 24s
2:	learn: 0.7000725	test: 0.7012394	best: 0.7012394 (2)	total: 11.6s	remaining: 1h 4m 15s
3:	learn: 0.7008954	test: 0.7015847	best: 0.7015847 (3)	total: 15.3s	remaining: 1h 3m 23s
4:	learn: 0.7013303	test: 0.7015847	best: 0.7015847 (3)	total: 20s	remaining: 1h 6m 14s
5:	learn: 0.7028737	test: 0.7029661	best: 0.7029661 (5)	total: 23.6s	remaining: 1h 5m 8s
6:	learn: 0.7033512	test: 0.7028126	best: 0.7029661 (5)	total: 27.3s	remaining: 1h 4m 30s
7:	learn: 0.7033555	test: 0.7027359	best: 0.7029661 (5)	total: 32.4s	remaining: 1h 7m
8:	learn: 0.7041613	test: 0.7042324	best: 0.7042324 (8)	total: 36.7s	remaining: 1h 7m 24s
9:	learn: 0.7079688	test: 0.7089137	best: 0.7089137 (9)	total: 40.9s	remaining: 1h 7m 28s
10:	learn: 0.7119596	test: 0.7137869	best: 0.7137869 (10)	total: 43.9s	remain

89:	learn: 0.7369916	test: 0.7437550	best: 0.7441004 (87)	total: 6m 55s	remaining: 1h 10m 6s
90:	learn: 0.7369617	test: 0.7439085	best: 0.7441004 (87)	total: 7m	remaining: 1h 10m 2s
91:	learn: 0.7371237	test: 0.7438318	best: 0.7441004 (87)	total: 7m 5s	remaining: 1h 9m 56s
92:	learn: 0.7370854	test: 0.7439085	best: 0.7441004 (87)	total: 7m 9s	remaining: 1h 9m 50s
93:	learn: 0.7371237	test: 0.7441388	best: 0.7441388 (93)	total: 7m 14s	remaining: 1h 9m 51s
94:	learn: 0.7371664	test: 0.7440236	best: 0.7441388 (93)	total: 7m 19s	remaining: 1h 9m 48s
95:	learn: 0.7374179	test: 0.7441388	best: 0.7441388 (93)	total: 7m 24s	remaining: 1h 9m 47s
96:	learn: 0.7373540	test: 0.7439469	best: 0.7441388 (93)	total: 7m 29s	remaining: 1h 9m 40s
97:	learn: 0.7374776	test: 0.7439853	best: 0.7441388 (93)	total: 7m 33s	remaining: 1h 9m 33s
98:	learn: 0.7376396	test: 0.7437934	best: 0.7441388 (93)	total: 7m 37s	remaining: 1h 9m 23s
99:	learn: 0.7377334	test: 0.7437550	best: 0.7441388 (93)	total: 7m 42s	rema

176:	learn: 0.7416219	test: 0.7476306	best: 0.7477457 (174)	total: 14m 29s	remaining: 1h 7m 24s
177:	learn: 0.7416517	test: 0.7480910	best: 0.7480910 (177)	total: 14m 34s	remaining: 1h 7m 18s
178:	learn: 0.7416731	test: 0.7479759	best: 0.7480910 (177)	total: 14m 39s	remaining: 1h 7m 15s
179:	learn: 0.7416773	test: 0.7479375	best: 0.7480910 (177)	total: 14m 44s	remaining: 1h 7m 8s
180:	learn: 0.7418095	test: 0.7481678	best: 0.7481678 (180)	total: 14m 47s	remaining: 1h 6m 57s
181:	learn: 0.7417669	test: 0.7482829	best: 0.7482829 (181)	total: 14m 52s	remaining: 1h 6m 49s
182:	learn: 0.7418095	test: 0.7482829	best: 0.7482829 (181)	total: 14m 56s	remaining: 1h 6m 41s
183:	learn: 0.7418223	test: 0.7483596	best: 0.7483596 (183)	total: 15m	remaining: 1h 6m 35s
184:	learn: 0.7418052	test: 0.7482445	best: 0.7483596 (183)	total: 15m 5s	remaining: 1h 6m 28s
185:	learn: 0.7418905	test: 0.7481678	best: 0.7483596 (183)	total: 15m 9s	remaining: 1h 6m 22s
186:	learn: 0.7419417	test: 0.7482061	best: 0.7

263:	learn: 0.7441417	test: 0.7488584	best: 0.7494340 (239)	total: 21m 21s	remaining: 59m 32s
264:	learn: 0.7441503	test: 0.7488201	best: 0.7494340 (239)	total: 21m 25s	remaining: 59m 25s
265:	learn: 0.7442441	test: 0.7487433	best: 0.7494340 (239)	total: 21m 31s	remaining: 59m 22s
266:	learn: 0.7442654	test: 0.7487050	best: 0.7494340 (239)	total: 21m 36s	remaining: 59m 19s
267:	learn: 0.7442952	test: 0.7487817	best: 0.7494340 (239)	total: 21m 42s	remaining: 59m 16s
268:	learn: 0.7442313	test: 0.7492038	best: 0.7494340 (239)	total: 21m 46s	remaining: 59m 10s
269:	learn: 0.7443123	test: 0.7488968	best: 0.7494340 (239)	total: 21m 51s	remaining: 59m 6s
270:	learn: 0.7443251	test: 0.7489352	best: 0.7494340 (239)	total: 21m 55s	remaining: 58m 59s
271:	learn: 0.7441929	test: 0.7492038	best: 0.7494340 (239)	total: 22m	remaining: 58m 54s
272:	learn: 0.7441758	test: 0.7491270	best: 0.7494340 (239)	total: 22m 4s	remaining: 58m 48s
273:	learn: 0.7442313	test: 0.7489736	best: 0.7494340 (239)	total:

...

615:	learn: 0.7509423	test: 0.7500480	best: 0.7510073 (562)	total: 48m 2s	remaining: 29m 56s
616:	learn: 0.7509508	test: 0.7500863	best: 0.7510073 (562)	total: 48m 6s	remaining: 29m 51s
617:	learn: 0.7509721	test: 0.7499328	best: 0.7510073 (562)	total: 48m 11s	remaining: 29m 47s
618:	learn: 0.7510617	test: 0.7500863	best: 0.7510073 (562)	total: 48m 16s	remaining: 29m 42s
619:	learn: 0.7510020	test: 0.7501631	best: 0.7510073 (562)	total: 48m 21s	remaining: 29m 38s
620:	learn: 0.7510446	test: 0.7501247	best: 0.7510073 (562)	total: 48m 25s	remaining: 29m 33s
621:	learn: 0.7510275	test: 0.7501631	best: 0.7510073 (562)	total: 48m 29s	remaining: 29m 28s
622:	learn: 0.7510275	test: 0.7500480	best: 0.7510073 (562)	total: 48m 35s	remaining: 29m 24s
623:	learn: 0.7509806	test: 0.7501631	best: 0.7510073 (562)	total: 48m 40s	remaining: 29m 20s
624:	learn: 0.7510190	test: 0.7501247	best: 0.7510073 (562)	total: 48m 45s	remaining: 29m 15s
625:	learn: 0.7510361	test: 0.7500096	best: 0.7510073 (562)	to

703:	learn: 0.7522043	test: 0.7504701	best: 0.7510073 (562)	total: 54m 23s	remaining: 22m 51s
704:	learn: 0.7522214	test: 0.7503933	best: 0.7510073 (562)	total: 54m 27s	remaining: 22m 47s
705:	learn: 0.7522342	test: 0.7502782	best: 0.7510073 (562)	total: 54m 32s	remaining: 22m 42s
706:	learn: 0.7522384	test: 0.7502782	best: 0.7510073 (562)	total: 54m 36s	remaining: 22m 37s
707:	learn: 0.7522811	test: 0.7505468	best: 0.7510073 (562)	total: 54m 41s	remaining: 22m 33s
708:	learn: 0.7523152	test: 0.7505084	best: 0.7510073 (562)	total: 54m 44s	remaining: 22m 28s
709:	learn: 0.7523152	test: 0.7503933	best: 0.7510073 (562)	total: 54m 49s	remaining: 22m 23s
710:	learn: 0.7524090	test: 0.7504317	best: 0.7510073 (562)	total: 54m 54s	remaining: 22m 18s
711:	learn: 0.7523450	test: 0.7505468	best: 0.7510073 (562)	total: 54m 58s	remaining: 22m 14s
712:	learn: 0.7524175	test: 0.7504701	best: 0.7510073 (562)	total: 55m 3s	remaining: 22m 9s
713:	learn: 0.7524388	test: 0.7505084	best: 0.7510073 (562)	to

791:	learn: 0.7533384	test: 0.7502782	best: 0.7510073 (562)	total: 1h 1m 12s	remaining: 16m 4s
792:	learn: 0.7533768	test: 0.7503549	best: 0.7510073 (562)	total: 1h 1m 18s	remaining: 16m
793:	learn: 0.7534024	test: 0.7502015	best: 0.7510073 (562)	total: 1h 1m 23s	remaining: 15m 55s
794:	learn: 0.7534578	test: 0.7501247	best: 0.7510073 (562)	total: 1h 1m 28s	remaining: 15m 51s
795:	learn: 0.7534323	test: 0.7499712	best: 0.7510073 (562)	total: 1h 1m 33s	remaining: 15m 46s
796:	learn: 0.7534493	test: 0.7499712	best: 0.7510073 (562)	total: 1h 1m 38s	remaining: 15m 41s
797:	learn: 0.7534323	test: 0.7499712	best: 0.7510073 (562)	total: 1h 1m 42s	remaining: 15m 37s
798:	learn: 0.7533896	test: 0.7500863	best: 0.7510073 (562)	total: 1h 1m 46s	remaining: 15m 32s
799:	learn: 0.7533726	test: 0.7501631	best: 0.7510073 (562)	total: 1h 1m 52s	remaining: 15m 28s
800:	learn: 0.7534749	test: 0.7501631	best: 0.7510073 (562)	total: 1h 1m 59s	remaining: 15m 24s
801:	learn: 0.7534792	test: 0.7500480	best: 0

877:	learn: 0.7544129	test: 0.7507387	best: 0.7510073 (562)	total: 1h 8m 16s	remaining: 9m 29s
878:	learn: 0.7543617	test: 0.7507003	best: 0.7510073 (562)	total: 1h 8m 23s	remaining: 9m 24s
879:	learn: 0.7544086	test: 0.7507003	best: 0.7510073 (562)	total: 1h 8m 27s	remaining: 9m 20s
880:	learn: 0.7543873	test: 0.7507770	best: 0.7510073 (562)	total: 1h 8m 33s	remaining: 9m 15s
881:	learn: 0.7543703	test: 0.7508154	best: 0.7510073 (562)	total: 1h 8m 39s	remaining: 9m 11s
882:	learn: 0.7543830	test: 0.7508921	best: 0.7510073 (562)	total: 1h 8m 44s	remaining: 9m 6s
883:	learn: 0.7544342	test: 0.7508921	best: 0.7510073 (562)	total: 1h 8m 51s	remaining: 9m 2s
884:	learn: 0.7544641	test: 0.7508538	best: 0.7510073 (562)	total: 1h 8m 55s	remaining: 8m 57s
885:	learn: 0.7544982	test: 0.7508154	best: 0.7510073 (562)	total: 1h 9m	remaining: 8m 52s
886:	learn: 0.7544598	test: 0.7508154	best: 0.7510073 (562)	total: 1h 9m 7s	remaining: 8m 48s
887:	learn: 0.7543489	test: 0.7507770	best: 0.7510073 (56

963:	learn: 0.7553935	test: 0.7510073	best: 0.7511224 (962)	total: 1h 15m 44s	remaining: 2m 49s
964:	learn: 0.7554575	test: 0.7509305	best: 0.7511224 (962)	total: 1h 15m 50s	remaining: 2m 45s
965:	learn: 0.7554447	test: 0.7509689	best: 0.7511224 (962)	total: 1h 15m 55s	remaining: 2m 40s
966:	learn: 0.7554532	test: 0.7508921	best: 0.7511224 (962)	total: 1h 16m	remaining: 2m 35s
967:	learn: 0.7555001	test: 0.7507387	best: 0.7511224 (962)	total: 1h 16m 6s	remaining: 2m 30s
968:	learn: 0.7555172	test: 0.7507387	best: 0.7511224 (962)	total: 1h 16m 11s	remaining: 2m 26s
969:	learn: 0.7555385	test: 0.7509689	best: 0.7511224 (962)	total: 1h 16m 18s	remaining: 2m 21s
970:	learn: 0.7555129	test: 0.7508538	best: 0.7511224 (962)	total: 1h 16m 24s	remaining: 2m 16s
971:	learn: 0.7555214	test: 0.7508154	best: 0.7511224 (962)	total: 1h 16m 28s	remaining: 2m 12s
972:	learn: 0.7555641	test: 0.7509305	best: 0.7511224 (962)	total: 1h 16m 34s	remaining: 2m 7s
973:	learn: 0.7555513	test: 0.7508921	best: 0.

In [19]:
y_pred=model.predict(X_val)
print(model.score(X_val, y_val))

0.7511223667549212


In [21]:
y_pred = y_pred.flatten()
y_pred

array([3, 2, 2, ..., 1, 2, 3], dtype=int64)

In [61]:
model.feature_importances_

array([7.49220737e+00, 2.63510626e+01, 1.62481296e+01, 2.63228215e+00,
       7.59758499e+00, 3.56940160e+00, 3.01992485e+00, 1.27170669e+00,
       5.98257220e+00, 3.44599423e+00, 5.04560840e+00, 3.94135404e+00,
       2.16861682e+00, 8.69255383e-01, 2.47297469e-01, 1.79905634e+00,
       2.66032186e-01, 3.97130240e-01, 7.27061158e-01, 1.22156555e+00,
       6.13224407e-01, 2.10643198e-01, 5.26295928e-02, 2.03995794e-01,
       2.00968358e-01, 1.01660721e+00, 2.33830990e+00, 8.11768918e-01,
       1.00401256e-01, 2.25655930e-02, 3.81118863e-02, 1.68544331e-02,
       2.95645634e-03, 1.97619967e-02, 1.70169134e-02, 2.94591809e-03,
       0.00000000e+00, 3.73943118e-02])

In [71]:
model.plot_tree(tree_idx=0,X_train)

SyntaxError: positional argument follows keyword argument (<ipython-input-71-11818178fb1c>, line 1)

## Prediccion Final y Submmit

In [23]:
predictions = model.predict(X_test)
predictions = predictions.flatten()
predictions = predictions.astype(int)
predictions

array([3, 2, 2, ..., 2, 2, 2])

In [25]:
output = pd.DataFrame({'building_id':test.building_id, 'damage_grade': predictions})
output.to_csv('my_submission.csv', index=False)

## Pickle

In [27]:
import pickle
#filename es el nombre que le daremos a nuestro archivo
filename = 'pickles/cat_boost_pickle.pkl'
#Con el comando dump y la instrucción wb nos aseguramos de crear y escribir sobre el 
#archivo creado, otra precaución es lo que deseamos guardar en este caso fue el modelo
pickle.dump(model, open(filename,'wb'))

In [56]:
loaded_model = pickle.load(open(filename, 'rb'))
model = loaded_model
loaded_model

In [76]:
X_train

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
142934,6,671,790,2,80,10,10,t,r,n,...,False,False,False,False,False,False,False,False,False,False
27635,17,930,2687,3,5,6,6,t,r,n,...,False,False,False,False,False,False,False,False,False,False
258386,6,1022,10827,2,40,9,6,t,r,n,...,False,False,False,False,False,False,False,False,False,False
246749,20,190,5803,2,20,4,5,t,w,q,...,False,False,False,False,False,False,False,False,False,False
113960,20,385,8073,1,15,8,3,o,r,n,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246607,6,1253,20,4,5,14,12,t,i,x,...,False,False,False,False,False,False,False,False,False,False
176548,6,376,1025,2,5,6,6,n,r,n,...,False,False,False,False,False,False,False,False,False,False
253333,10,488,3201,2,5,4,4,t,r,n,...,False,False,False,False,False,False,False,False,False,False
179545,20,943,8270,2,15,7,7,t,r,n,...,False,False,False,False,False,False,False,False,False,False


## Pruebo con otros features

In [85]:
cate_features_index = np.where(X.dtypes != float)[0]
cate_features_index

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37], dtype=int64)

In [86]:

from catboost import Pool, CatBoostClassifier, cv

model = CatBoostClassifier(iterations=5000,eval_metric='Accuracy', loss_function='MultiClass',use_best_model=True,random_seed=1).fit(X_train,y_train,cat_features=cate_features_index,eval_set=(X_val,y_val))
print(model.score(X_train, y_train))

Learning rate set to 0.065122
0:	learn: 0.6940181	test: 0.6956372	best: 0.6956372 (0)	total: 3.61s	remaining: 5h 45s
1:	learn: 0.6989213	test: 0.6992825	best: 0.6992825 (1)	total: 6.42s	remaining: 4h 27m 37s
2:	learn: 0.6988019	test: 0.6987069	best: 0.6992825 (1)	total: 9.74s	remaining: 4h 30m 20s
3:	learn: 0.7004349	test: 0.7010475	best: 0.7010475 (3)	total: 12.9s	remaining: 4h 28m 2s
4:	learn: 0.7022128	test: 0.7035417	best: 0.7035417 (4)	total: 16.1s	remaining: 4h 27m 25s
5:	learn: 0.7023493	test: 0.7025057	best: 0.7035417 (4)	total: 19.7s	remaining: 4h 32m 54s
6:	learn: 0.7024218	test: 0.7018533	best: 0.7035417 (4)	total: 22.6s	remaining: 4h 28m 6s
7:	learn: 0.7031039	test: 0.7025057	best: 0.7035417 (4)	total: 25.8s	remaining: 4h 27m 50s
8:	learn: 0.7030101	test: 0.7026591	best: 0.7035417 (4)	total: 28.6s	remaining: 4h 24m 15s
9:	learn: 0.7028481	test: 0.7018917	best: 0.7035417 (4)	total: 31.6s	remaining: 4h 23m 12s
10:	learn: 0.7031764	test: 0.7025057	best: 0.7035417 (4)	total: 34

88:	learn: 0.7330903	test: 0.7403400	best: 0.7403400 (88)	total: 5m 27s	remaining: 5h 53s
89:	learn: 0.7333120	test: 0.7403016	best: 0.7403400 (88)	total: 5m 31s	remaining: 5h 1m 47s
90:	learn: 0.7332694	test: 0.7403400	best: 0.7403400 (88)	total: 5m 35s	remaining: 5h 2m 2s
91:	learn: 0.7334442	test: 0.7404167	best: 0.7404167 (91)	total: 5m 40s	remaining: 5h 2m 26s
92:	learn: 0.7334101	test: 0.7406853	best: 0.7406853 (92)	total: 5m 43s	remaining: 5h 2m 16s
93:	learn: 0.7335167	test: 0.7406853	best: 0.7406853 (92)	total: 5m 47s	remaining: 5h 2m 40s
94:	learn: 0.7335977	test: 0.7407237	best: 0.7407237 (94)	total: 5m 52s	remaining: 5h 2m 54s
95:	learn: 0.7336190	test: 0.7409923	best: 0.7409923 (95)	total: 5m 55s	remaining: 5h 2m 48s
96:	learn: 0.7335593	test: 0.7405318	best: 0.7409923 (95)	total: 5m 59s	remaining: 5h 3m 5s
97:	learn: 0.7336957	test: 0.7411074	best: 0.7411074 (97)	total: 6m 4s	remaining: 5h 3m 54s
98:	learn: 0.7337426	test: 0.7413760	best: 0.7413760 (98)	total: 6m 8s	remai

175:	learn: 0.7380660	test: 0.7452132	best: 0.7453283 (174)	total: 11m 32s	remaining: 5h 16m 25s
176:	learn: 0.7380660	test: 0.7453666	best: 0.7453666 (176)	total: 11m 37s	remaining: 5h 16m 37s
177:	learn: 0.7381257	test: 0.7451364	best: 0.7453666 (176)	total: 11m 41s	remaining: 5h 16m 50s
178:	learn: 0.7381044	test: 0.7451748	best: 0.7453666 (176)	total: 11m 45s	remaining: 5h 16m 52s
179:	learn: 0.7380660	test: 0.7453666	best: 0.7453666 (176)	total: 11m 49s	remaining: 5h 16m 45s
180:	learn: 0.7381385	test: 0.7453666	best: 0.7453666 (176)	total: 11m 53s	remaining: 5h 16m 29s
181:	learn: 0.7382664	test: 0.7454818	best: 0.7454818 (181)	total: 11m 58s	remaining: 5h 16m 56s
182:	learn: 0.7382962	test: 0.7454434	best: 0.7454818 (181)	total: 12m 2s	remaining: 5h 17m
183:	learn: 0.7384412	test: 0.7455969	best: 0.7455969 (183)	total: 12m 6s	remaining: 5h 16m 54s
184:	learn: 0.7384753	test: 0.7456352	best: 0.7456352 (184)	total: 12m 11s	remaining: 5h 17m 10s
185:	learn: 0.7384241	test: 0.745827

261:	learn: 0.7412808	test: 0.7475538	best: 0.7476306 (257)	total: 17m 34s	remaining: 5h 17m 44s
262:	learn: 0.7412339	test: 0.7474387	best: 0.7476306 (257)	total: 17m 38s	remaining: 5h 17m 36s
263:	learn: 0.7413107	test: 0.7475538	best: 0.7476306 (257)	total: 17m 42s	remaining: 5h 17m 38s
264:	learn: 0.7414428	test: 0.7474003	best: 0.7476306 (257)	total: 17m 46s	remaining: 5h 17m 44s
265:	learn: 0.7414087	test: 0.7473620	best: 0.7476306 (257)	total: 17m 50s	remaining: 5h 17m 33s
266:	learn: 0.7414940	test: 0.7475154	best: 0.7476306 (257)	total: 17m 55s	remaining: 5h 17m 38s
267:	learn: 0.7415878	test: 0.7475154	best: 0.7476306 (257)	total: 17m 59s	remaining: 5h 17m 38s
268:	learn: 0.7415324	test: 0.7476689	best: 0.7476689 (268)	total: 18m 3s	remaining: 5h 17m 39s
269:	learn: 0.7416176	test: 0.7477840	best: 0.7477840 (269)	total: 18m 8s	remaining: 5h 17m 42s
270:	learn: 0.7415707	test: 0.7477840	best: 0.7477840 (269)	total: 18m 11s	remaining: 5h 17m 33s
271:	learn: 0.7415793	test: 0.74

346:	learn: 0.7433657	test: 0.7482445	best: 0.7488968 (340)	total: 23m 34s	remaining: 5h 16m 4s
347:	learn: 0.7434553	test: 0.7482829	best: 0.7488968 (340)	total: 23m 39s	remaining: 5h 16m 12s
348:	learn: 0.7434340	test: 0.7485515	best: 0.7488968 (340)	total: 23m 43s	remaining: 5h 16m 8s
349:	learn: 0.7434382	test: 0.7481294	best: 0.7488968 (340)	total: 23m 48s	remaining: 5h 16m 13s
350:	learn: 0.7434979	test: 0.7480910	best: 0.7488968 (340)	total: 23m 52s	remaining: 5h 16m 14s
351:	learn: 0.7435107	test: 0.7482445	best: 0.7488968 (340)	total: 23m 57s	remaining: 5h 16m 16s
352:	learn: 0.7434681	test: 0.7482445	best: 0.7488968 (340)	total: 24m 2s	remaining: 5h 16m 23s
353:	learn: 0.7435278	test: 0.7483212	best: 0.7488968 (340)	total: 24m 6s	remaining: 5h 16m 21s
354:	learn: 0.7435704	test: 0.7482829	best: 0.7488968 (340)	total: 24m 9s	remaining: 5h 16m 11s
355:	learn: 0.7436130	test: 0.7482445	best: 0.7488968 (340)	total: 24m 13s	remaining: 5h 15m 54s
356:	learn: 0.7437239	test: 0.74839

...

4623:	learn: 0.7747975	test: 0.7527723	best: 0.7530409 (4438)	total: 5h 46m 30s	remaining: 28m 10s
4624:	learn: 0.7747932	test: 0.7526956	best: 0.7530409 (4438)	total: 5h 46m 35s	remaining: 28m 6s
4625:	learn: 0.7747719	test: 0.7527340	best: 0.7530409 (4438)	total: 5h 46m 39s	remaining: 28m 1s
4626:	learn: 0.7747676	test: 0.7528107	best: 0.7530409 (4438)	total: 5h 46m 43s	remaining: 27m 57s
4627:	learn: 0.7747676	test: 0.7528107	best: 0.7530409 (4438)	total: 5h 46m 47s	remaining: 27m 52s
4628:	learn: 0.7747165	test: 0.7528491	best: 0.7530409 (4438)	total: 5h 46m 51s	remaining: 27m 47s
4629:	learn: 0.7747506	test: 0.7528491	best: 0.7530409 (4438)	total: 5h 46m 56s	remaining: 27m 43s
4630:	learn: 0.7747463	test: 0.7528107	best: 0.7530409 (4438)	total: 5h 47m	remaining: 27m 38s
4631:	learn: 0.7747506	test: 0.7528491	best: 0.7530409 (4438)	total: 5h 47m 4s	remaining: 27m 34s
4632:	learn: 0.7747506	test: 0.7528491	best: 0.7530409 (4438)	total: 5h 47m 8s	remaining: 27m 29s
4633:	learn: 0.774

4707:	learn: 0.7751855	test: 0.7528875	best: 0.7532328 (4698)	total: 5h 52m 35s	remaining: 21m 52s
4708:	learn: 0.7751940	test: 0.7528107	best: 0.7532328 (4698)	total: 5h 52m 39s	remaining: 21m 47s
4709:	learn: 0.7751940	test: 0.7528107	best: 0.7532328 (4698)	total: 5h 52m 44s	remaining: 21m 43s
4710:	learn: 0.7752281	test: 0.7528875	best: 0.7532328 (4698)	total: 5h 52m 48s	remaining: 21m 38s
4711:	learn: 0.7752537	test: 0.7528491	best: 0.7532328 (4698)	total: 5h 52m 52s	remaining: 21m 34s
4712:	learn: 0.7752707	test: 0.7528875	best: 0.7532328 (4698)	total: 5h 52m 57s	remaining: 21m 29s
4713:	learn: 0.7752963	test: 0.7530026	best: 0.7532328 (4698)	total: 5h 53m 1s	remaining: 21m 25s
4714:	learn: 0.7753049	test: 0.7529642	best: 0.7532328 (4698)	total: 5h 53m 5s	remaining: 21m 20s
4715:	learn: 0.7753134	test: 0.7530026	best: 0.7532328 (4698)	total: 5h 53m 10s	remaining: 21m 16s
4716:	learn: 0.7753304	test: 0.7530409	best: 0.7532328 (4698)	total: 5h 53m 14s	remaining: 21m 11s
4717:	learn:

4791:	learn: 0.7758335	test: 0.7532712	best: 0.7533095 (4736)	total: 5h 58m 40s	remaining: 15m 34s
4792:	learn: 0.7758506	test: 0.7532712	best: 0.7533095 (4736)	total: 5h 58m 45s	remaining: 15m 29s
4793:	learn: 0.7758591	test: 0.7531944	best: 0.7533095 (4736)	total: 5h 58m 49s	remaining: 15m 25s
4794:	learn: 0.7758719	test: 0.7531944	best: 0.7533095 (4736)	total: 5h 58m 53s	remaining: 15m 20s
4795:	learn: 0.7758549	test: 0.7531944	best: 0.7533095 (4736)	total: 5h 58m 57s	remaining: 15m 16s
4796:	learn: 0.7758762	test: 0.7533095	best: 0.7533095 (4736)	total: 5h 59m 2s	remaining: 15m 11s
4797:	learn: 0.7758634	test: 0.7533095	best: 0.7533095 (4736)	total: 5h 59m 6s	remaining: 15m 7s
4798:	learn: 0.7758932	test: 0.7534247	best: 0.7534247 (4798)	total: 5h 59m 10s	remaining: 15m 2s
4799:	learn: 0.7758932	test: 0.7533479	best: 0.7534247 (4798)	total: 5h 59m 15s	remaining: 14m 58s
4800:	learn: 0.7758847	test: 0.7533863	best: 0.7534247 (4798)	total: 5h 59m 19s	remaining: 14m 53s
4801:	learn: 0

4876:	learn: 0.7762812	test: 0.7532328	best: 0.7534630 (4802)	total: 6h 4m 44s	remaining: 9m 11s
4877:	learn: 0.7763025	test: 0.7532328	best: 0.7534630 (4802)	total: 6h 4m 49s	remaining: 9m 7s
4878:	learn: 0.7763196	test: 0.7531561	best: 0.7534630 (4802)	total: 6h 4m 54s	remaining: 9m 2s
4879:	learn: 0.7762855	test: 0.7530793	best: 0.7534630 (4802)	total: 6h 4m 58s	remaining: 8m 58s
4880:	learn: 0.7763281	test: 0.7530026	best: 0.7534630 (4802)	total: 6h 5m 2s	remaining: 8m 53s
4881:	learn: 0.7763068	test: 0.7530409	best: 0.7534630 (4802)	total: 6h 5m 6s	remaining: 8m 49s
4882:	learn: 0.7762983	test: 0.7530409	best: 0.7534630 (4802)	total: 6h 5m 11s	remaining: 8m 45s
4883:	learn: 0.7762770	test: 0.7530793	best: 0.7534630 (4802)	total: 6h 5m 15s	remaining: 8m 40s
4884:	learn: 0.7762727	test: 0.7530409	best: 0.7534630 (4802)	total: 6h 5m 20s	remaining: 8m 36s
4885:	learn: 0.7763324	test: 0.7530026	best: 0.7534630 (4802)	total: 6h 5m 24s	remaining: 8m 31s
4886:	learn: 0.7763196	test: 0.752

4961:	learn: 0.7768312	test: 0.7530793	best: 0.7534630 (4802)	total: 6h 10m 48s	remaining: 2m 50s
4962:	learn: 0.7768696	test: 0.7531177	best: 0.7534630 (4802)	total: 6h 10m 52s	remaining: 2m 45s
4963:	learn: 0.7768440	test: 0.7531177	best: 0.7534630 (4802)	total: 6h 10m 55s	remaining: 2m 41s
4964:	learn: 0.7768099	test: 0.7532712	best: 0.7534630 (4802)	total: 6h 11m	remaining: 2m 36s
4965:	learn: 0.7768227	test: 0.7532712	best: 0.7534630 (4802)	total: 6h 11m 5s	remaining: 2m 32s
4966:	learn: 0.7768739	test: 0.7532328	best: 0.7534630 (4802)	total: 6h 11m 9s	remaining: 2m 27s
4967:	learn: 0.7768398	test: 0.7532712	best: 0.7534630 (4802)	total: 6h 11m 13s	remaining: 2m 23s
4968:	learn: 0.7768611	test: 0.7532712	best: 0.7534630 (4802)	total: 6h 11m 17s	remaining: 2m 18s
4969:	learn: 0.7768526	test: 0.7533863	best: 0.7534630 (4802)	total: 6h 11m 22s	remaining: 2m 14s
4970:	learn: 0.7768781	test: 0.7533095	best: 0.7534630 (4802)	total: 6h 11m 26s	remaining: 2m 10s
4971:	learn: 0.7769080	tes

In [87]:
y_pred=model.predict(X_val)
print(model.score(X_val, y_val))

0.7536548866121792


In [89]:
predictions = model.predict(X_test)
predictions = predictions.flatten()
predictions = predictions.astype(int)
predictions

array([3, 2, 2, ..., 2, 2, 2])

In [90]:
output = pd.DataFrame({'building_id':test.building_id, 'damage_grade': predictions})
output.to_csv('my_submission.csv', index=False)

In [91]:
#filename es el nombre que le daremos a nuestro archivo
filename = 'pickles/cat_boost_5000_i_pickle.pkl'
#Con el comando dump y la instrucción wb nos aseguramos de crear y escribir sobre el 
#archivo creado, otra precaución es lo que deseamos guardar en este caso fue el modelo
pickle.dump(model, open(filename,'wb'))

In [92]:
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model